## Create figures from the code

In [1]:
import glob
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import MeanShift

import sys
sys.path.append('..')

from modules.gait_metrics import *
import modules.numpy_funcs as nf
import modules.plotting as pl


In [2]:
%matplotlib auto

Using matplotlib backend: TkAgg


In [3]:
file_name = '2014-12-08_P004_Post_000'

In [11]:
best_pos_dir = os.path.join('..', 'data', 'kinect', 'best pos')
hypo_dir = os.path.join('..', 'data', 'kinect', 'processed', 'hypothesis')

hypo_paths = glob.glob(os.path.join(hypo_dir, '*.pkl'))
best_pos_paths = glob.glob(os.path.join(best_pos_dir, '*.pkl'))

hypo_paths = [x for x in hypo_paths if file_name in x]
best_pos_paths = [x for x in best_pos_paths if file_name in x]

df_hypo = pd.read_pickle(hypo_paths[0])
df_best_pos = pd.read_pickle(best_pos_paths[0])

In [12]:
df_head_feet = df_best_pos

In [28]:
# Convert all position vectors to float type
# so they can be easily input to linear algebra functions
df_head_feet = df_head_feet.applymap(pd.to_numeric)

# Cluster frames with mean shift to locate the walking passes
frames = df_head_feet.index
mean_shift = MeanShift(bandwidth=60).fit(nf.to_column(frames))
labels = mean_shift.labels_

# Sort labels so that the frames are in temporal order
labels = nf.map_to_whole(labels)

# DataFrames for each walking pass in a trial
pass_dfs = list(nf.group_by_label(df_head_feet, labels))

In [32]:
plt.figure()

points = np.column_stack([frames, np.ones(len(frames))])
pl.scatter_labels(points, labels)

plt.show()

In [30]:
df_pass = pass_dfs[0]

## Foot Distance

In [15]:
foot_dist = (df_pass.L_FOOT - df_pass.R_FOOT).apply(norm)

plt.figure()

pl.scatter_series(foot_dist, c='k', s=15)
plt.plot(foot_dist, c='k', linewidth=0.7)

plt.xlabel('Frame')
plt.ylabel('Foot Distance [cm]')

plt.show()

In [16]:
_, direction_pass = direction_of_pass(df_pass)

# Assign correct sides to feet
df_pass = asi.assign_sides_pass(df_pass, direction_pass)

# Ensure there are no missing frames in the walking pass
df_pass = pf.make_index_consecutive(df_pass)
df_pass = df_pass.applymap(lambda x: x if isinstance(x, np.ndarray)
                           else np.full(3, np.nan))

In [17]:
foot_series = df_pass.R_FOOT
frames_pass = df_pass.index.values

In [18]:
foot_points = np.stack(foot_series)

step_signal = lin.line_coordinate_system(np.zeros(3), direction_pass,
                                         foot_points)

is_stance = pde.detect_phases(step_signal)

## Step Signal

In [20]:
plt.figure()

points = np.column_stack((frames_pass, step_signal))
pl.scatter_labels(points, ~is_stance)
plt.xlabel('Frame')
plt.ylabel('Y', rotation=0)
plt.legend(['stance', 'swing'])

plt.show()